<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

大規模なゲノムデータを用いた解析: Genomic Prediction その1
---

　今回から、大規模なゲノムデータセットを用いた解析の中でGenomicPredictionを勉強・体験していきます。

大規模ゲノムデータ解析シリーズ: 
- GWAS: Genome wide association study(GWAS)
- **Genomic Prediction入門 : Genomic Predictionとは**, 機械学習の基本, 統計モデル, 育種への応用...等々

※今回のテキストはPythonになります。

# 今回の勉強内容
　このテキストでは、そのGenomic Prediction入門の第1回として、以下を勉強します。

1. Genomic Predictionとは？
  1. Genomic Predictionの概要
  1. Genomic Predictionの応用
1. Genomic Predictionの解析手順・必要な知識等
  1. データの準備
  1. 予測モデルの構築
  1. 構築したモデルの応用
1. Genomic Predictionを体験してみる
  1. 理想的な遺伝子型を探してみる
  1. 最適な交配組み合わせを探してみる

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################

## プログラム・サンプルファイルのダウンロード
!wget -q -O 'GP_linear_data.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GP_linear_data.csv
!wget -q -O 'GP_linear_data2.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GP_linear_data2.csv
!wget -q -O 'GP_sample_genotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GP_sample_genotype.csv
!wget -q -O 'GP_sample_phenotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GP_sample_phenotype.csv
!wget -q -O genomic_prediction.py https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/scripts/genomic_prediction_1.py?raw=true
## 確認
!ls | grep -e 'GP_linear_data.csv' -e 'GP_linear_data2.csv' -e 'GP_sample_genotype.csv' -e 'GP_sample_phenotype.csv' -e 'genomic_prediction.py'

# 1. Genomic Predictionとは？

これまで扱ってきたQTL-seqやGWASという手法は、形質に影響を与えている原因遺伝子を探るためにQTLを探索する手法です。

具体的には、異なる系統を交配することでゲノムがシャッフルされた分離集団を作成したり、遺伝子型が異なる系統を集めて、遺伝的背景が異なる集団を作成し、シーケンスデータと形質値データを集めて解析することで、形質値に関与していそうな遺伝領域を探る手法でした。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gene_mapping.png?raw=true" alt="gene_mapping" height="280px" align="middle">

これらの手法は、「形質に関与している原因遺伝子の特定」というのが大きな目的でした。

遺伝子を同定してその機能を調べていくという研究において非常に重要なアプローチの一つになります。

しかし、実際に育種を進めていく上で更に重要になってくるのは、
* 形質に影響を与える遺伝子がそれぞれどのくらい効果を持っているのか？
* その遺伝子に変異が入ったり、別の系統と交配し遺伝子型が変わることで、最終的にどのような形質値になるのか？
* 多数の遺伝子に支配されており、効果が分散されている場合、それぞれの遺伝子の効果をどう捉えるか？

といった点です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_gene_effect.png?raw=true" alt="GP_gene_effect" height="500px" align="middle">

この様な問題を解決しようとする手法の1つがGenomic Prediction(ゲノミック予測)になります。

Genomic Predictionは「ある遺伝子型を持つ生物がどんな形質値になるのか」を予測するアプローチになります。

全体のゲノム領域から形質値を予測するため、特定の遺伝子だけでなく、ゲノム領域全体の総合的な効果を考慮する様な手法になります。

## 1-1. Genomic Predictionの概要
　Genomic Predictionは「ある遺伝子型がどのくらいの形質値になるのか」を予測するアプローチということでした。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_image.png?raw=true" alt="GP_image" height="200px" align="middle">

　上図の様に、作物の遺伝子型を調べて、その情報から作物がどのように育っていくのか予測することが出来れば、育種において非常に有用な情報となります。

　この「予測モデル」を構築していくのがGenomic Predictionで最も重要な過程になります。

　予測モデルの構築には、遺伝子型と形質値の関係性をうまく解釈するのに十分な量の遺伝子型と形質の情報が必要になります。

　そのため、これまで扱ってきたQTL-seqやGWASよりも更に多くのデータセットが必要になってきます。

　大規模な交配集団等の準備とその遺伝子型の調査(Genotyping)、及び形質値の測定(Phenotyping)が必要になり、高コストな研究の１つです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_data.png?raw=true" alt="GP_data" height="400px" align="middle">

　初回講義でもありましたが、近年シーケンスの技術が向上しコストが低下してきたことで様々な作物で予測モデルを構築する試みが可能になってきました。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_moore.png?raw=true" alt="GP_moore" height="280px" align="middle">

　Genomic Predictionでは、統計学的な知識・技術を活用して、得られた大規模なデータの遺伝子型と形質値の関係性を上手く説明する様な予測モデルを構築し、そのモデルを用いて関心のある系統等の遺伝子型から形質値を予測していく、という流れになります。

## 1-2. Genomic Predictionの応用

　それでは、良いGenomic Predictionモデルの構築に成功し、「ある遺伝子型がどのくらいの形質値になるのか」を予測することが出来るとどのようなことに活用できるのでしょうか。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_image.png?raw=true" alt="GP_image" height="200px" align="middle">


### 1-2-1. 最適な遺伝子型の考案

正確に遺伝子型から形質値を予測するモデルを構築することが出来れば、目的に応じた遺伝子型を推定することが出来ます。

目的に応じてモデルを構築することで、
* 高収量の系統
* 病気や低/高温ストレスに強い系統
* 低肥料で効率よく収量が得られる系統

等、様々な目的に最適な遺伝子型を推定することが出来ます。

詳細は次回以降扱いますが、大まかに2つの方法があります。

* ランダムに遺伝子型を何千~何億通りシミュレートして、各遺伝子型に対しGenomic Predictionモデルで形質値を予測。その結果から理想的な遺伝子型を選び出す。
* 構築したGenomic Predictionモデルを解釈し、その情報に基づいて推定する。
  * `形質値 = 遺伝子A × 2000 + 遺伝子 B × -1800`みたいな関係性が分かれば、遺伝子Aを持っていて、遺伝子Bに変異が入っている遺伝子型が望ましい、のような形ですね。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_ideal.png?raw=true" alt="GP_ideal" height="180px" align="middle">

### 1-2-2. Genomic Selection
　予測モデルの活用例として最も代表的なものは、Genomic Selectionになります。

　分野によってはGenomic SelectionとGenomic Predictionを同じものとして扱っているところもあります。

　植物育種の世界だと、遺伝子型から形質値を予測することが出来れば、どんな組み合わせで掛け合わせると最適な系統を作ることが出来るのか、予測することが可能になります。

1. 手持ちの系統をシーケンスして遺伝子型を調べる。
2. 関心のある系統を交配した結果、どんな遺伝子型の系統が生まれるかシミュレーションする。
3. シミュレーションしたそれぞれの遺伝子型から、Genomic Predictionモデルで形質値を予測する。
4. この結果を踏まえ、目的に応じた最適な組み合わせを選択する。

「確率は低くても、最も収量が増える系統を作りたい」とか、「小規模の交配集団で収量を少し増加させる系統を作りたい。」など。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_GS.png?raw=true" alt="GP_GS" height="400px" align="middle">

　従来の育種では収量の高い系統同士をかけあわせる等、形質値に基づいて交配組み合わせを考えていましたが、予測モデルに基づいて様々な組み合わせでどのような系統を作ることが出来るのか考えることで、より最適な組み合わせが見つかる可能性があります。

　また、育成にコストや時間のかかる畜産の世界では、生まれてきた子供がどの程度遺伝的な能力を持っているか評価するために用いられています。

そのため、これまではゲノム育種は特に毒物の育種の世界で非常に重要なアプローチの１つとなっており、様々なGenomic Selectionのモデルが考案されていました。

※植物を対象としたゲノム育種と、動物を対象としたゲノム育種には栽培状況・生育環境など異なっていることがたくさんあるため、出来ることや目的に大きな違いがあります。


## (参考)1-3. 植物と動物の違い

動物(畜産)を対象としたGenomic Prediction(Selection)と植物(作物)を対象としたGenomic Prediction(Selection)には様々な違いがあります。

| |  動物  |  植物  |
| ---- | ---- | ---- |
| 交配 | ヘテロが残る、時間がかかる | (自殖作物なら)固定化可能、Speed breedingが可能 |
|集団サイズ| 小さい | 多様、基本的に大きい |
|環境の影響| 形質による | 形質による |
|遺伝的変異| 意図的に入れるのは難しい | 交配や実験的に容易に入れられる |
|共有や保存| 共有や保存は難しい | 種として共有・保存が容易 |
| 価格 |  1個体あたり大  |  1個体あたり小  |
| GSによる早期選抜 | 利益が大きい | 利点はそれほど |
| 投資リターン |  投資に対するリターンが大きい  |  投資に対するリターンが小さい  |

当然ながら実験的に扱いやすいのは植物です。多様な遺伝的変異を持つ集団を作ったり、理想的な遺伝子型を構築した際に、それを実際に作出するのは植物の方が容易なため、良いGenomic Predictionモデルを構築したり、応用するのも容易になります。

一方、動物の場合は大きな集団サイズを用意するのが難しかったり、形質の調査にも時間がかかる等、難しい点があります。しかし、上手く選抜や改良を進めることが出来れば経済的な視点で見ると大きなリターンがあるというのも動物の特徴です。

(参考)
* [Genomic prediction unifies animal and plant breeding
programs to form platforms for biological discovery](https://www.nature.com/articles/ng.3920) (Hickey et al., 2017, Nature genetics)
* [Enhancing Genetic Gain through Genomic
Selection: From Livestock to Plants](https://doi.org/10.1016/j.xplc.2019.100005) (Xu et al., 2019, Plant Communications)

# 2. Genomic Predictionの解析手順・必要な知識等

実際に解析の流れを説明しながら体験していきましょう。今回はイネを対象としたGenomic Predictionを考えてみます。

大まかな流れとしては以下の様になります。
1. データの準備(集団の準備、Genotyping、Phenotyping)
2. 予測モデルの構築
3. 構築したモデルの応用

## 2-1. データの準備

　上述してある通り、良いGenomic Predictionモデルを構築するためには、大規模なデータが必要になります。

　また、ただ単純に色んな系統をかき集めてくるのではなく、計画的に交配を行うことで、良いモデルを作りやすい集団を準備することもできます。

　例えばGWASの時に説明したRIL集団という集団もその一つです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_RIL.png?raw=true" alt="GWAS_RIL" height="320px">

RIL集団は親系統の遺伝子型がランダムに混ざってホモ接合型で固定されているため、ヘテロ接合の影響(雑種強勢など)を考慮する必要が無く、GWASなどを用いたQTLの探索に非常に強力な材料でした。

他にもMAGIC(Multi-parent Advanced Generation Inter-Cross)集団と呼ばれる複数品種からなる多系交雑集団などもあったりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_magic.png?raw=true" alt="GP_magic" height="320px">

figure: from [Bandillo et al., 2013](https://doi.org/10.1186/1939-8433-6-11)

今回の実習では、NAM(Nested Association Mapping)集団という交配集団を使用します。

NAM集団は、片親の系統を固定し、もう片親に様々な系統を使用してRIL集団を作成したものになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_NAM.png?raw=true" alt="GP_NAM" height="320px">

この様な集団を作成することで、ある1つの系統に、様々な系統の遺伝子型がランダムに混ざった様な集団が作成できます。

この図で言うと「ひとめぼれ」系統に様々な系統の遺伝子型が混ざった様な集団になります。

準備には時間とコストがかかりますが、ひとめぼれ系統の収量を増加させたり、改良を進める際には非常に強力な材料となります。

今回はこのNAM集団の遺伝子型データと形質値データを使ってGenomic Predictionを行っていきます。

データとしては前回のGWASで使ったものと似たような形になります。

In [ ]:
# Genotype data
import pandas as pd
import numpy as np
genotype = pd.read_csv("GP_sample_genotype.csv")
genotype

In [ ]:
# Phenotype data
phenotype = pd.read_csv("GP_sample_phenotype.csv")
phenotype

## 2-2. 予測モデルの構築

材料となる集団を作成し、形質値の調査(Phenotyping)や遺伝子型の調査(Genotyping)を終え、次はそれらのデータを用いて予測モデルを構築していきます。

「予測モデル」と言ってもピンとこないかもしれないので、極簡単な例で説明します。

### 2-2-1. 簡単な回帰モデルによる予測モデル

例えば、ある１箇所の遺伝子座の遺伝子型によって形質値が決まっている形質があったとします。

この時、遺伝子座の遺伝子型毎に形質を予測したい場合は以下の様に回帰式から予測することが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_sample_model.png?raw=true" alt="GP_sample_model" height="320px">

遺伝子型を調べて、AAなのかABなのかBBなのかが分かれば、予測式からy(形質値)を計算して予測できる、という訳ですね。

簡単なサンプルデータで例を確認してみましょう。

In [ ]:
# sample dataを見る
sample_data = pd.read_csv("GP_linear_data.csv")
sample_data

このような各個体の形質値とある遺伝子座の遺伝子型データがあったとします。

このデータから形質値を遺伝子型から予測する回帰式($y=S\beta + \epsilon$)を作ってみましょう。

Pythonには`scikit-learn`というライブラリがあり、回帰分析や機械学習など様々な解析を行うことが出来ます。

(詳細は省きます)

In [ ]:
# 線形回帰式
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

geno = pd.DataFrame(sample_data.Genotype)
pheno = pd.DataFrame(sample_data.Phenotype)
model = LinearRegression()
model.fit(geno, pheno)
preds = model.predict(geno)

print(f"y = S×{model.coef_[0][0]:.3f}+{model.intercept_[0]:.3f}")
plt.scatter(geno, pheno)
plt.plot(geno, preds, color = '#000000', linewidth=2)
plt.xticks([0, 1, 2],  ['AA', 'AB', 'BB'])
plt.show()

形質値が遺伝子型がAA(0)の時は50.941、AB(1)の時は23.436+50.941、BB(2)の時は2x23.436+50.941になると予測できる式が出来た訳ですね。

この時、作成した予測モデル(y = S×23.436+50.941)がどれくらい正しいか(モデルの予測精度)を確認してみます。

予測精度の指標として実際の形質値(実測値)と、モデル式から計算された形質の予測値を比較し、相関係数を求めてみます。

相関係数は0~1の範囲をとり、1に近いほど、予測値と実測値が相関していることを示す指標になります。

In [ ]:
# 予測値
predicted_values = model.predict(geno)
sample_data["Predicted"] = predicted_values
display(sample_data)

plt.scatter(sample_data["Phenotype"], sample_data["Predicted"])
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(sample_data["Phenotype"], sample_data["Predicted"])[0, 1]))

### (参考)線形回帰で得られる直線

　上のコードでは、y = S×23.436+50.941という式がPythonのライブラリによって計算されましたが、

回帰モデルによって得られる直線はどのような直線かというと、

　得られたデータについて、実測値と予測値(回帰直線上の点)のずれ（残差）が小さければデータの説明変数(今回だと遺伝子型)と目的変数(形質値)の関係を"うまく"表現している直線と言えるでしょう。

　今回おこなった単回帰モデルでは、残差の二乗の合計値、**残差平方和 (residual sum of squares)** が最も小さくなる係数 $\beta$ と誤差 $\epsilon$ の直線を求めています。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3eIFmh8PDRx64eFArwdgxO2CGt3PEi272ny1dyqAMue0un_yL_GMgZ0CsyvBnX4lEC9BfOEdfTNGsiEG-R4xZDPM9zMHwHcINcnQFxcdTmSgsF7LotLsBpwzs0S49fZtN1fQrbHY7JrB9m2kwuDGb9r=w815-h560-no?authuser=0" alt="least squares" height="180px">

$$ 残差平方和: \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 = \sum_{i=1}^{N} (S_i\beta + \epsilon - y_i)^2 $$

  - $S_{i}$: i番目のデータの遺伝子型の値
  - $y_{i}$: i番目のデータの形質値
  - $\hat{y}_{i}$: i番目のデータの形質値の予測値(回帰直線上の値)
  - $N=1,2,3,...,n$: データの個数

　残差平方和のように、最小化を目指す関数のことを、**目的関数 objective function** や **コスト関数 cost function** と呼びます。  

　殆どの機械学習の各アルゴリズムは、何らかの目的関数を持っています。データに基づいて、目的関数の最小化（ときには最大化）をおこなってパラメータの値を求めていきます(今回だと$\beta$や$\epsilon$)。どのような目的関数を選択するかも良いモデルを作成するためには非常に重要です。

<small>※ 線形回帰モデルでパラメータ（係数や誤差）の最適値を求める方法は、「係数$\beta$ と 誤差$e$ の最適値をどうやって求めているか？」を参照してください。</small>






#### 係数$\beta$ と 誤差$\epsilon$ の最適値をどうやって求めているか？

　scikit-learnの `LinearRegression()` は、**最小二乗法 least squares method**と呼ばれる方法を使って、残差平方和が最小値（極小値）になる係数$\beta$ と誤差$\epsilon$ を求めています。最小二乗法の概要は次のとおりです。

1. 目的関数（残差平方和）
$$ \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 = \sum_{i=1}^{N} (S_i\beta + \epsilon - y_i)^2 $$
  - $S_{i}$: i番目のデータの説明変数の値
  - $y_{i}$: i番目のデータの目的変数の値（実測値）
  - $\hat{y}_{i}$: i番目のデータの目的変数の値（予測値）
  - $N=1,2,3,...,n$: データの個数
  
1. 目的関数の$\sum$の部分を展開すると、
$$ \sum_{i=1}^{N} (\beta S_i + \epsilon - y_i)^2 = \beta^{2}\sum_{i=1}^{N}{S_{i}^{2}} + N\epsilon^2 + \sum_{i=1}^{N}{y_{i}^{2}} - 2\beta\sum_{i=1}^{N}{S_{i}y_{i}} + 2\beta \epsilon\sum_{i=1}^{N}{S_{i}} -2\epsilon\sum_{i=1}^{N}{y_{i}} $$

1. 上の展開式を$\beta$の関数として考えると、下に凸な二次関数であることがわかります。
$$ \sum_{i=1}^{N} (\beta S_i + \epsilon - y_i)^2 = \Bigl(\sum_{i=1}^{N}{S_{i}^{2}} \Bigr) \beta^{2} - \Bigl( 2\sum_{i=1}^{N}{S_{i}y_{i}} + 2 \epsilon\sum_{i=1}^{N}{S_{i}} \Bigr) \beta + \Bigl( N\epsilon^2 + \sum_{i=1}^{N}{y_{i}^{2}}　-2\epsilon\sum_{i=1}^{N}{y_{i}} \Bigr) $$
同様に、$\epsilon$の関数として考えた場合も下に凸な二次関数となります。したがって、残差平方和が最小となる$\beta$と$\epsilon$は、両者の関数が同時に極小値となるときの値と一致しそうです。

1. そこで、$\beta$や$\epsilon$についての偏微分関数が0になる値（極小値）を調べます。
 $$ (\beta について偏微分)=0 $$
 $$ (\epsilon について偏微分)=0 $$
1. 上の2つの方程式を解くと、目的関数が極小値となる$\beta$や$e$の値が求まります


<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/least_squares_method.png?raw=true" alt="least squares method" height="180px">


### 2-2-2. 重回帰式

さて、先ほどの例では、たった１つの遺伝子座の遺伝子型によって形質が決まっている場合でした。

しかし基本的に、たった１つの遺伝子で決定される形質というのはあまりありません。大抵複数の遺伝子が関与しています。

そこで次は、少し増やして３箇所の遺伝子座の遺伝子型によって形質値が決まっている形質を考えてみましょう。

この時には、先ほどの単回帰分析とは異なり、複数の説明変数を回帰式に入れた重回帰分析を行います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_sample_model2.png?raw=true" alt="GP_sample_model2" height="200px">

簡単なサンプルデータで例を確認してみましょう。

In [ ]:
# sample data
sample_data = pd.read_csv("GP_linear_data2.csv")
sample_data

In [ ]:
# 線形回帰式
model = LinearRegression()
model = model.fit(sample_data.iloc[:, :3], sample_data.iloc[:, 3])
print(f"y = SNP1×{model.coef_[0]:.3f}+SNP2×{model.coef_[1]:.3f}+SNP3×{model.coef_[2]:.3f}+{model.intercept_:.3f}")

このように、3か所のSNPの遺伝子型が分かれば、形質値が予測できる式が出来ました。

実際の形質値と作成した回帰式で予測した形質値がどれくらい近いのかをグラフにして、回帰モデルの予測精度を見てみます。

In [ ]:
# 予測
y_preds = model.predict(sample_data.iloc[:, :3])
plt.scatter(sample_data.iloc[:, 3], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(sample_data["Phenotype"], y_preds)[0, 1]))

今回のサンプルデータは１つや３つの遺伝子型で形質が決まっているシンプルな場合だったので回帰式で上手く形質値と遺伝子型の関係性を表すことが出来ました。

このように、回帰式の様な形で、形質値と遺伝子型の関係性を表すことで、遺伝子型の情報が得られれば、形質値を予測できるようになります。

そのため、もし実際の遺伝子型と形質値のデータでも、良い予測モデルを作ることが出来れば、遺伝子型の情報を調べるだけでどんな形質値になるか予測することが出来る様になるわけです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_image.png?raw=true" alt="GP_image" height="180px" align="middle">

しかし、より多数の遺伝子に支配されている場合には、回帰式では形質値と遺伝子型の関係性を説明しきれなかったり、

基本的にある形質がどの遺伝子によってコントロールされているかは殆ど分かっていないので、予測モデルにどの場所の遺伝子型の情報を入れるべきかを決めるのはとても難しいです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_unknown.png?raw=true" alt="GP_unknown" height="250px" align="middle">

### 2-2-3. NAM集団のデータで回帰モデル

では実際のNAM集団のデータを元に回帰モデルで予測式を作ってみましょう。

今回は手に入った6366個のSNPの遺伝子型全ての情報を使って重回帰モデルを作り、予測モデルを構築してみます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_sample_model3.png?raw=true" alt="GP_sample_model3" height="80px">

In [ ]:
# 線形回帰
model = LinearRegression()
model = model.fit(genotype.iloc[:, 2:].T, phenotype["GN_mean"])
equation = "y = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

In [ ]:
# 予測精度
y_preds = model.predict(genotype.iloc[:, 2:].T)
plt.scatter(phenotype["GN_mean"], y_preds)
print("相関係数: {}".format(np.corrcoef(phenotype["GN_mean"], y_preds)[0, 1]))

## 2-3. 構築したモデルの応用

それでは重回帰モデルを使用して構築した予測モデルを使って、育種に活用していきましょう。

と行きたいところですが、実はこの予測モデル、一見凄く予測精度が高いモデルに見えますが、様々な問題点が残っています。

(今回の課題でどんな問題があるか考えてみてください。次回以降解説します。)

今日は、まともなやり方で構築したGenomic Predictionモデルをこちらで用意したので、それを使って予測モデルの活用例をいくつか試してみましょう。



# 3. Genomic Predictionモデルの応用を体験してみる

今回は、NAM集団のデータを元に、籾数を予測するGenomic Predictionモデルを構築したので、そのモデルを利用して以下の2点をやってみましょう。

* 理想的な遺伝子型を探してみる
* 最適な交配組み合わせを探してみる

In [ ]:
# 予測モデルの構築
import warnings
warnings.filterwarnings('ignore')
from genomic_prediction import split_dataset
from genomic_prediction import make_genomic_prediction_model
from genomic_prediction import check_equation
test_genotype, test_phenotype, train_genotype, train_phenotype = split_dataset(genotype, phenotype, "GN_mean", test=0.2)
GN_prediction_model = make_genomic_prediction_model(train_genotype, train_phenotype, "GN_mean")
print("\nPrediction model is ...")
print(check_equation("GN_mean", GN_prediction_model), "\n")

## 3-1. 理想的な遺伝子型を探してみる

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_ideal.png?raw=true" alt="GP_ideal" height="150px" align="middle">

下のコードを動かすと任意のゲノム領域の遺伝子型がひとめぼれ系統からREXMONTという系統に変わった時に、どんな形質値になるかを計算してくれます。

変化させる領域を色々変えて籾数が最も大きくなるような遺伝子型を試してみましょう。

---

`regions = [ [領域1], [領域2], ... [領域x] ]`という形で複数の領域を指定することもできます。

例えば、`[['chr01', 12000000, 20000000], ['chr05', 0, 5000000], ['chr12', 10000000, 15000000]]`と書くと、

* chr01の12000000~20000000bpの領域
* chr05の0~5000000bpの領域
* chr12の10000000~15000000bpの領域

がREXMONTに変わった時の値を計算してくれます。

' 'や , の付け忘れ、chr1ではなくchr01、最初と最後の[]の付け忘れなどに注意して色々試してみてください。

※各染色体の長さは長いもので40000000bpくらい、短いと20000000bpくらいです。


In [ ]:
# 任意の場所の遺伝子型が別の系統に変わるとどんな形質が出来るか
from genomic_prediction import predict_customized_genotype
regions = [['chr01', 10000000, 20000000], ['chr03', 10000000, 20000000]]
predict_customized_genotype(genotype, regions, GN_prediction_model)

## 3-2. 最適な交配組み合わせを探してみる

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_GS.png?raw=true" alt="GP_GS" height="300px" align="middle">

NAM_001 ~ NAM_747までの747系統の中でどれとどれを交配させるとより良い系統を作ることが出来るでしょうか？

下のコードを動かすと任意の2系統を交配し、F2集団を作った時にどんな形質値の集団になるかを計算してくれます。

`NAM_001`と`NAM_002`の部分を色々変えて試してみましょう。

In [ ]:
# 組み合わせでどんな形質が出来るか
from genomic_prediction import predict_progeny_phenotype
predict_progeny_phenotype("NAM_001", "NAM_002", 100, genotype, GN_prediction_model)

# まとめ

　今回、Genomic Predictionの概要と解析の流れ、応用例をみてきました。
- Genomic Prediction: 
  - 「ある遺伝子型を持つ生物がどんな形質値になるのか」を予測するアプローチ
  - 理想的な遺伝子型や最適な交配組み合わせを決めるのに役立つ

次回からは予測モデル構築に関してより具体的な進め方や様々な統計モデル・手法の紹介などを学んでいきます。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>